## Import Modules

In [1]:
import Tools
import Dataset as ds

import numpy as np
import torch
from ultralytics import settings
from ultralytics import YOLO


## Load Dataset

In [2]:
# Base paths for the images and labels
train_images_path = '../datasets/AVOIDDS/images/train'
train_labels_path = '../datasets/AVOIDDS/labels/train'
val_images_path = '../datasets/AVOIDDS/images/valid'
val_labels_path = '../datasets/AVOIDDS/labels/valid'

# Base path for metadata
metadata_path = '../datasets/AVOIDDS/metadata.json'

# Create the DataFrames for the train and validation sets
train_df = ds.create_dataframe(train_images_path, train_labels_path, metadata_path)
valid_df = ds.create_dataframe(val_images_path, val_labels_path, metadata_path)

#train_df.info()

# Create sub datasets

In [ ]:
# TEST - cessna only dataset test

#test_train_df = train_df[(train_df['ac'] == 'Cessna Skyhawk') & (train_df['weather'] == 0)]
#test_valid_df = valid_df[(valid_df['ac'] == 'Cessna Skyhawk') & (valid_df['weather'] == 0)]
#
#dataset_name = 'cessna_only'
#
#ds.create_sub_dataset(dataset_name, test_train_df, test_valid_df)

# Create Augmented Datasets

In [ ]:
test_dataset_dir = "datasets/cessna_only"

"""
Methods: 
 - flip
 - rotate
 - bnc
 - histEq
 - whiteBal
 - sharpen
 - gaussian
 - zoom

"""

augmentation_metadata = {
    'methods': {
        'hsv': {
            'parameters': {
                'type': 'h',  # Probability of applying the augmentation
                'shift': 50,
                'p': 1.0
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        },
        'flip': {
            'parameters': {
                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        },
        'rotate': {
            'parameters': {
                'angle': 90,  # Rotation angle
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.1  # 30% of the training images
        },
        'bnc': {
            'parameters': {
                'alpha': 1.5,  # contrast control
                'beta': 50, # brightness control
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        },
        'histEq': {
            'parameters': {
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        },
        'whiteBal': {
            'parameters': {
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        },
        'sharpen': {
            'parameters': {
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        },
        'gaussian': {
            'parameters': {
                'var_limit': (10.0, 50.0) # range of noise varaince
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        },
        'zoom': {
            'parameters': {
                'zoom_factor': 3  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.1  # 50% of the training images
        }       
    }
}

#ds.augment_dataset(test_dataset_dir, augmentation_metadata)

# Test Augmentation Methods

In [ ]:
# TEST - block to test aug methods

"""
Methods: 
 - flip
 - rotate
 - bnc
 - gaussian
 - histEq
 - whiteBal
 - sharpen
 - zoom
 - hsv
"""

image = '21'
dataset = 'train-aug'
aug_type = 'flip'


test_image_path = f'datasets/cessna_only/images/{dataset}/{image}.jpg'
test_label_path = f'datasets/cessna_only/labels/{dataset}/{image}.txt'

aug_image_path = f'datasets/cessna_only/images/{dataset}/{image}-{aug_type}.jpg'
aug_label_path = f'datasets/cessna_only/labels/{dataset}/{image}-{aug_type}.txt'

#Tools.overlay_bbox_image(aug_image_path, aug_label_path)

# Yolo Config

In [3]:
# Ultralytics settings
settings.reset()

settings.update({'datasets_dir': f'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets'})
#settings.update({'weights_dir': f'C:/github/Third-Year-Project/Intruder-Aircraft-Detection/weights'})
#settings.update({'runs_dir': f'C:/github/Third-Year-Project/Intruder-Aircraft-Detection/runs'})

print(settings)


# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")


""" Disable Data Augmentation (potentially)
hsv_h: 0.0  # hue
hsv_s: 0.0  # saturation
hsv_v: 0.0  # value
degrees: 0.0  # rotation
translate: 0.0  # translate
scale: 0.0  # scale
shear: 0.0  # shear
perspective: 0.0  # perspective
flipud: 0.0  # flip up-down
fliplr: 0.0  # flip left-right
mosaic: 0.0  # mosaic
mixup: 0.0  # mixup
"""





{'settings_version': '0.0.4', 'datasets_dir': 'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets', 'weights_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
True


' Disable Data Augmentation (potentially)\nhsv_h: 0.0  # hue\nhsv_s: 0.0  # saturation\nhsv_v: 0.0  # value\ndegrees: 0.0  # rotation\ntranslate: 0.0  # translate\nscale: 0.0  # scale\nshear: 0.0  # shear\nperspective: 0.0  # perspective\nflipud: 0.0  # flip up-down\nfliplr: 0.0  # flip left-right\nmosaic: 0.0  # mosaic\nmixup: 0.0  # mixup\n'

# Test Parameters

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

_, test_train_df = train_test_split(
    train_df,
    test_size=500,  # Number of items you want in your sample
    stratify=train_df['ac'],  # Stratify based on the combined column
    random_state=42  # Ensures reproducibility
)

_, test_val_df = train_test_split(
    valid_df,
    test_size=100,  # Number of items you want in your sample
    stratify=valid_df['ac'],  # Stratify based on the combined column
    random_state=42  # Ensures reproducibility
)


dataset_name = 'test500'

ds.create_sub_dataset(dataset_name, test_train_df, test_val_df)

Removing dataset if pre-existing
Copying training files:


Copying files:   0%|          | 0/1000 [00:00<?, ?it/s]

Copying validation files:


Copying files:   0%|          | 0/200 [00:00<?, ?it/s]

Dataset 'test500' created at C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets


In [5]:
augmentation_metadata = {
    'methods': {        
        'flip': {
            'parameters': {
                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.5  # 50% of the training images
        }        
    }
}


ds.augment_dataset(f'../datasets/{dataset_name}', augmentation_metadata)

Removing existing directories/files:   0%|          | 0/3 [00:00<?, ?it/s]

Copying files:   0%|          | 0/500 [00:00<?, ?it/s]

Copying files:   0%|          | 0/500 [00:00<?, ?it/s]

Applying augmentations:   0%|          | 0/250 [00:00<?, ?it/s]

An error occurred: Expected y_max for bbox (0.36195600000000006, 0.7324, 0.764684, 1.000886, 0) to be in the range [0.0, 1.0], got 1.000886.


# Training

In [6]:
# train on non-augemnted
#dataset_name = 'coco128'
#augmentation_metadata = None

epochs = 20
dataset_name='test500'
dataset_path=f'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets\\{dataset_name}\\{dataset_name}.yaml'
project =  f'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\logs\\{dataset_name}\\pure' #save_dir # weight save path

# Ultralytics settings

print(settings) 



model = YOLO('yolov8n.pt')
# train model
results = model.train(data=dataset_path, epochs=epochs, device=0, project=project)

# validate model
#metrics = model.val()





{'settings_version': '0.0.4', 'datasets_dir': 'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets', 'weights_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\test500.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\train... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1846.08it/s]


train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\valid... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 484.06it/s]

val: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\valid.cache


...

      20/20      2.15G      1.204      1.246      1.171          3        640: 100%|██████████| 32/32 [00:03<00:00,  8.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.19it/s]

                   all        100        100      0.652       0.66      0.647      0.314



20 epochs completed in 0.037 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train\weights\best.pt...
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.29it/s]


                   all        100        100      0.702       0.65      0.668      0.335
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\pure\train


In [7]:
# train on augmented

epochs = 20
dataset_name='test500'
dataset_path=f'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets\\{dataset_name}\\{dataset_name}-aug.yaml'
project =  f'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\logs\\{dataset_name}\\augmented' #save_dir # weight save path

# Ultralytics settings

print(settings) 



model = YOLO('yolov8n.pt')
# train model
results = model.train(data=dataset_path, epochs=epochs, device=0, project=project)

# validate model
#metrics = model.val()


{'settings_version': '0.0.4', 'datasets_dir': 'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets', 'weights_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\test500-aug.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:\github\Third-Year-Project\Intruder-Aircraft-Detection\

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\train-aug... 749 images, 0 backgrounds, 0 corrupt: 100%|██████████| 749/749 [00:00<00:00, 943.09it/s]


train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\train-aug.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\test500\labels\valid.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]


Plotting labels to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      2.38G      1.202      1.085      1.129         13        640: 100%|██████████| 47/47 [00:05<00:00,  7.93it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.56it/s]

                   all        100        100       0.78       0.68      0.715      0.346



20 epochs completed in 0.045 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train\weights\best.pt...
Ultralytics YOLOv8.1.45  Python-3.11.9 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.70it/s]


                   all        100        100      0.721      0.697       0.71       0.37
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\test500\augmented\train
